<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_%D0%9C%D0%BD%D0%BE%D0%B3%D0%BE%D1%81%D0%BB%D0%BE%D0%B9%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (это рукописные цифры поэтому будет 10 классов выходов)
Получите качество на тестовой выборке не ниже 88%
Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв,
количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.'''

'Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (это рукописные цифры поэтому будет 10 классов выходов)\nПолучите качество на тестовой выборке не ниже 88%\nИзменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв,\nколичества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.'

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import torchvision as tv
import pandas as pd
import time

In [3]:
BATCH_SIZE=256

train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True) #train=True - загружается train датасет, transform=tv.transforms.ToTensor() -  картинки преобразуются в тензоры, download=True - они будут скачиваться
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 303kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.52MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.15MB/s]


In [4]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [5]:
test_dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: .
    Split: Test
    StandardTransform
Transform: ToTensor()

In [15]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Dropout(0.25),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(128, 10)
)

In [7]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=.001)
num_epochs = 20

In [13]:
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()

        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)

        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)

        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )
        if (test_acc / test_passed) > 0.88:
            print("Цель достигнута ->",test_acc / test_passed)
            break


In [14]:
train_model()

ep: 0, taked: 13.244, train_loss: 0.3471443142028565, train_acc: 0.8743666666666666, test_loss: 0.38188057448714974, test_acc: 0.86
ep: 1, taked: 13.616, train_loss: 0.3254081350057683, train_acc: 0.8825833333333334, test_loss: 0.3607105538249016, test_acc: 0.8674
ep: 2, taked: 13.744, train_loss: 0.3105020576335014, train_acc: 0.8867166666666667, test_loss: 0.3722747839987278, test_acc: 0.8655
ep: 3, taked: 13.968, train_loss: 0.298595592696616, train_acc: 0.8901666666666667, test_loss: 0.3559592517092824, test_acc: 0.8676
ep: 4, taked: 14.137, train_loss: 0.28888761566040366, train_acc: 0.8932666666666667, test_loss: 0.3655526785179973, test_acc: 0.8658
ep: 5, taked: 14.978, train_loss: 0.2721366510112235, train_acc: 0.9002833333333333, test_loss: 0.3509956904686987, test_acc: 0.8739
ep: 6, taked: 14.103, train_loss: 0.2665804105870267, train_acc: 0.9015833333333333, test_loss: 0.3676395708695054, test_acc: 0.8674
ep: 7, taked: 14.080, train_loss: 0.26237366053652256, train_acc: 0.90